In [1]:
!pip install albumentations==0.4.6
!pip install segmentation_models_pytorch
!pip install torchmetrics
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65152 sha256=05a2b3465a6119d5d06261f3953e17a717019ddeaa6b54530311f5651e97f2ee
  Stored in directory: /root/.cache/pip/wheels/f9/d7/0c/6ed42fd872f7d1af78b25045f8b16be330f2c70ae72c83e37d
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22

In [2]:
#utils
import torch
import os
import random
import numpy as np
from pathlib import Path
from typing import List
from sklearn.model_selection import train_test_split
import collections
from collections import defaultdict
from functools import reduce

#viz
import matplotlib.pyplot as plt

#images
from skimage.io import imread
import albumentations as albu
from albumentations.pytorch import ToTensor
from albumentations.augmentations.transforms import MotionBlur

#torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
import ssl
from tqdm import tqdm
from PIL import Image
ssl._create_default_https_context = ssl._create_unverified_context

from torchmetrics.functional import dice
import monai.metrics
from natsort import natsorted

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root = Path("/content/drive/MyDrive/ETVProject/ETV_Project/dataset_v2_fom_cp_cropped/")
test_chiari_images = Path("/content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images")
test_chiari_masks = Path("/content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/masks")
#test_chiari_masks = None

In [5]:
test_images_chiari = sorted(test_chiari_images.glob("*.png"))
test_images_chiari = natsorted(test_images_chiari)
test_masks_chiari = None
#test_images_chiari

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda


In [7]:
color_dict = {'background': [  0,   0,   0],
          'lpca':[  0,   0, 128], # BLUE 163 occurances
          'rmb':[  0, 128,   0],  # GREEN 182 occurances
          'cp':[  0, 128, 128],   # CYAN 38 occurances
          'fom':[128,   0,   0],  # RED 46 occurances
          'rpca':[128,   0, 128], # PURPLE 229 occurances
          'lmb':[128, 128,   0],  # YELLOW 173 occurances
          'ca':[128, 128, 128]}   # GRAY 87 occurances
colors = list(color_dict.values())

In [8]:
def encode_mask(mask, colors = colors):
  semantic_map = []
  for color in colors:
    equality = np.equal(mask, color)
    class_map = np.all(equality, axis = -1)
    semantic_map.append(class_map)

  semantic_map = np.stack(semantic_map, axis = -1)

  return semantic_map.astype('uint8')

In [9]:
def reverse_transform(inp):
  inp = inp.cpu().numpy().transpose((1, 2, 0))
  inp = np.clip(inp, 0, 1)
  inp = (inp * 255).astype(np.uint8)

  return inp

In [10]:
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.
    # Arguments
        image: The one-hot format image

    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified
        class key.
    """
    x = np.argmax(reverse_transform(image), axis = -1)
    return x

In [11]:
def colour_code_segmentation(image):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values

    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(colors)
    # print(colour_codes)
    # print(image.astype(int))
    # print(np.unique(image.astype(int)))
    x = colour_codes[image.astype(int)]

    return x

In [12]:

netClass = smp.Unet
modelDict = {
    "encoder_name": "resnet152",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    "encoder_weights": "imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    "in_channels": 3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    "classes": 8,                      # model output channels (number of classes in your dataset)
    "activation":'softmax2d',# TO DO activation function softmax2d
    # "decoder_attention_type":"scse"
}

net = netClass(**modelDict)
model = netClass(**modelDict)
model2 = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:01<00:00, 237MB/s]


In [13]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()
    # print(pred)
    intersection = (pred * target).sum(dim=2).sum(dim=2)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

    return loss.mean()


In [14]:
def calc_loss(loss_fn, pred, target, metrics, bce_weight=0.5):
    bce = F.cross_entropy(pred, target)

    # pred = torch.sigmoid(pred)
    dice = dice_loss(pred, target)
    # print(type(pred[0]), type(target[0]))
    # print(pred.dtype, target.long())
    # print(pred.size(), target.size())
    # dice = loss_fn(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)
    # loss = dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    return loss

In [15]:
def plot_side_by_side(img_arrays, count, batch_size, save_location):
    flatten_list = reduce(lambda x,y: x+y, zip(*img_arrays))

    plot_img_array(np.array(flatten_list),count, batch_size,save_location, ncol=len(img_arrays))

def plot_img_array(img_array, count, batch_size, save_location, ncol=3 ):
    nrow = len(img_array) // ncol
    f, plots = plt.subplots(nrow, ncol, sharex='all', sharey='all', figsize=(ncol * 4, nrow * 4), squeeze = False)
    for i in range(len(img_array)):
        plots[i // ncol, i % ncol]
        plots[i // ncol, i % ncol].imshow(img_array[i])
    plt.savefig(save_location +str(count)  + '.jpg')

In [16]:
def predict_images_test(loader, batch_size, model, save_location):
    i = 1
    epoch_samples = 0
    dice = 0
    metrics = defaultdict(float)
    count = 1
    saver = 1
    phase1_idx = phase2_idx = phase3_idx = 0
    flag1 = flag2 = flag3 = False
    fps = 2
    limit = fps * 2

    phase1_count = phase2_count = 0
    with torch.no_grad():
        for data_dict in tqdm(loader):
            inputs = data_dict["image"].to(device)
            predicted = model(inputs)

            for i in range(len(predicted)):
              predicted_image = Image.fromarray(colour_code_segmentation(reverse_one_hot(predicted[i])).astype('uint8'), 'RGB')

              colors = predicted_image.getcolors()
              converted_colors = []
              for color in colors:
                if(color[1] != (0, 0, 0)):
                  converted_colors.append(color[1])
              pixdata = predicted_image.load()
              for y in range(predicted_image.size[1]):
                  for x in range(predicted_image.size[0]):
                      if (0, 0, 128) in converted_colors:
                        if pixdata[x,y] != (0, 0, 128) and pixdata[x, y] != (0, 128, 0) and pixdata[x, y] != (128, 128, 0) and pixdata[x, y] != (128, 0, 128):
                          pixdata[x, y] = (0, 0, 0)
                      else:
                        if pixdata[x,y] != (128, 0, 0) and pixdata[x, y] != (0, 128, 128):
                          pixdata[x, y] = (0, 0, 0)
                      if (0, 128, 128) not in converted_colors:
                        if pixdata[x, y]==(128, 0, 0):
                          pixdata[x, y] = (0, 0, 0)

              if (128, 0, 0) in converted_colors and (0, 128, 128) in converted_colors and flag1 == False:
                flag1 = True
                phase1_idx = count

              if (flag1 == True and flag2 == False):
                if (128, 0, 0) not in converted_colors and (0, 128, 128) not in converted_colors:
                  phase1_count+=1
                  if phase1_count>=limit:
                    flag2 = True
                    phase2_idx = count
                else:
                  phase1_count = 0

              if flag1==True and flag2==True and flag3 == False:
                if (128, 128, 0) in converted_colors and (0, 128, 0) in converted_colors:
                  phase2_count+=1
                  if phase2_count>=limit:
                    flag3 = True
                    phase3_idx = count
                else:
                  phase2_count = 0


              encoded_mask = encode_mask(predicted_image)
              encoded_mask = encoded_mask.transpose(2, 0, 1)
              predicted[i] = torch.Tensor(encoded_mask)

            count+=1

            #print("Image number", count)
            #image.show()

            #print(predicted)
            #print(type(predicted))

            input_images_rgb = [reverse_transform(x) for x in inputs.cpu()]
            predicted_rgb = [colour_code_segmentation(reverse_one_hot(predicted_mask)) for predicted_mask in predicted.cpu()]

            for rgbImage, inputImage in zip(predicted, input_images_rgb):

                rgbImagePIL = Image.fromarray(colour_code_segmentation(reverse_one_hot(rgbImage)).astype('uint8'), 'RGB')
                inputImagePIL = Image.fromarray(inputImage)

                #rgbImagePIL.save("/content/drive/MyDrive/ETVProject/ETV_Project/predictions/" + str(saver) + "_predicted_mask.png")
                #inputImagePIL.save("/content/drive/MyDrive/ETVProject/ETV_Project/predictions/" + str(saver) + "_input.png")

                overlay_img = Image.blend(inputImagePIL, rgbImagePIL, 0.5)

                overlay_img.save("/content/drive/MyDrive/ETVProject/ETV_Project/chiari_predictions3/" + str(saver) + "_masked_frame.png")

                saver += 1


            #print(predicted_rgb)
            #print(type(predicted_rgb))
            #print("Length ", predicted.shape)
            #for image in predicted_rgb:
              #plt.savefig(save_location + '/image'+str(i)  + '.jpg')
              #i += 1
            #plot_side_by_side([predicted_rgb], count, batch_size, save_location)


    return (phase1_idx, phase2_idx, phase3_idx)

In [17]:
class SegmentationDataset(Dataset):
  def __init__(self, images:List[Path], masks:List[Path] = None, transform = None) -> None:
    self.images = images
    self.masks = masks
    self.transform = transform

  def __len__(self) -> int:
    return len(self.images)

  def __getitem__(self, idx:int) -> dict:
    image_path = self.images[idx]
    image = imread(self.images[idx])
    result = {"image":image}
    if self.masks is not None:
      mask = imread(self.masks[idx])
      mask = mask[:,:,0:3]
      result["mask"] = encode_mask(mask)
    if self.transform is not None:
      result = self.transform(**result)
        #** is used for variable number of args
    if self.masks is not None:
      result['mask'] = torch.ceil(result['mask'][0])
      result['mask'] = torch.swapaxes(torch.swapaxes(result['mask'], 0, 2), 1, 2)
      result["mask_no_color"] = reverse_one_hot(result["mask"])

    result["filename"] = image_path.name
    return result
# train_dataset = SegmentationDataset(all_images, all_masks, train_transforms)
# train_dataset[0]

In [18]:
BORDER_CONSTANT = 0
def pre_transforms(crop_size=500, image_size=300):
    # return []
    # return [albu.CenterCrop(480, 640, p=1), albu.Resize(300, 300, p=1)]
    return [albu.Resize(224, 224, p=1)]

# Probability = 0.5
def augmentation_transforms():
    return [
            albu.VerticalFlip(p=0.5),
            albu.RandomRotate90(p=0.5),
            albu.Blur(10, p = 0.2),
            albu.CLAHE(p=0.8),
            albu.RandomBrightness(limit = 0.3,p = 0.5),
            albu.RandomContrast(limit = 0.2, p = 0.7),
            albu.RandomGamma(p=0.8),
    ]

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    # return [albu.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), ToTensor()] # TO DO use image net normalize [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    return [ToTensor()] # This also divides image & mask by 255 # TO DO use image net normalize [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

def compose(transforms_to_compose):
    # combine all augmentations into single pipeline
    result = albu.Compose([
      item for sublist in transforms_to_compose for item in sublist
    ])
    return result


In [19]:
aug = augmentation_transforms()

# image_transforms = compose([
#     pre_transforms(),
#     aug['imageOnly'],
# ])
# both_transforms = compose([
#     pre_transforms(),
#     aug['both'],
#     post_transforms(),
# ])
train_transforms = compose([
    pre_transforms(),
    augmentation_transforms(),
    post_transforms(),

])
train_transforms2 = compose([
    pre_transforms(),
    post_transforms(),

])
valid_transforms = test_transforms = compose([pre_transforms(),  post_transforms()])
# valid_transforms = test_transforms = compose([post_transforms()])


# show_transforms = compose([pre_transforms(), augmentation_transforms(), post_transforms()])
# show_transforms = compose([post_transforms()])

In [20]:
test_dataset_chiari = SegmentationDataset(test_images_chiari, test_masks_chiari, test_transforms)

In [21]:


loader_test_video_chiari = DataLoader(
    test_dataset_chiari,
    batch_size=1,
    shuffle = False,
    num_workers=2,
    drop_last=True
)

In [22]:
print("Testing for Batch size 2")
model2_loaded = netClass(**modelDict)
model2_loaded = model2_loaded.to(device)
model2_loaded.load_state_dict(torch.load('/content/drive/MyDrive/ETVProject/ETV_Project/dataset_v2_fom_cp_cropped/model_bs2_random_try.pt', map_location=device))
model2_loaded.eval()
idxs = predict_images_test(loader_test_video_chiari, 1, model2_loaded, '/content/drive/MyDrive/ETVProject/ETV_Project/Testvideo1_predictions3')
phase1_idx, phase2_idx, phase3_idx = idxs

Testing for Batch size 2


100%|██████████| 319/319 [03:13<00:00,  1.65it/s]


In [23]:
!pip install ultralytics==8.0.20
import ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [24]:
from ultralytics import YOLO
from tqdm import tqdm
from PIL import Image

In [25]:
import os.path
from natsort import natsorted
imlist = natsorted(os.listdir("/content/drive/MyDrive/ETVProject/ETV_Project/chiari_predictions3"))


In [26]:
model = YOLO('/content/drive/MyDrive/ETVProject/ETV_Project/detect/train/weights/best.pt')

In [27]:
from pathlib import Path

In [28]:
source = '/content/drive/MyDrive/ETVProject/ETV_Project/chiari_predictions3'

In [29]:
results=[]
for im in tqdm(imlist):
  if '.png' in im:
    #print(source+'/'+im)
    results.append(model(source+'/'+im))

  0%|          | 0/319 [00:00<?, ?it/s]Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
100%|██████████| 319/319 [00:06<00:00, 46.09it/s]


In [30]:
results

[[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
  type: <class 'torch.Tensor'>
  shape: torch.Size([0, 6])
  dtype: torch.float32
   + tensor([], device='cuda:0', size=(0, 6))],
 [Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
  type: <class 'torch.Tensor'>
  shape: torch.Size([0, 6])
  dtype: torch.float32
   + tensor([], device='cuda:0', size=(0, 6))],
 [Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
  type: <class 'torch.Tensor'>
  shape: torch.Size([0, 6])
  dtype: torch.float32
   + tensor([], device='cuda:0', size=(0, 6))],
 [Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
  type: <class 'torch.Tensor'>
  shape: torch.Size([0, 6])
  dtype: torch.float32
   + tensor([], device='cuda:0', size=(0, 6))],
 [Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
  type: <class 'torch.Tensor'>
  shape: torch.Size([0, 6])
  dtype: torch.float32
   + tensor([], device='cuda:0', 

In [31]:
fps = 2
limit = fps * 3
count = 0
phase4_count = phase5_count = phase4_idx = phase5_idx = phase6_idx = phase6_count = 0
flag4 = flag5 = flag6 = False

for i in tqdm(range(len(results))):
  if i <= phase3_idx:
    count+=1
    continue
  if flag4 == True and flag5 == False and 0.0 in results[i][0].boxes.cls and (results[i][0].boxes.conf[0] > 0.5 or (len(results[i][0].boxes.conf)>1 and results[i][0].boxes.conf[1] > 0.5)):
    phase5_count = phase5_count + 1
    if phase5_count>=limit:
      flag5 = True
      phase5_idx = count
  elif flag4 == True and flag5 == False and 0.0 not in results[i][0].boxes.cls:
    phase5_count = 0

  if flag4 == False and flag5 == False and 1.0 in results[i][0].boxes.cls and (results[i][0].boxes.conf[0] > 0.5 or (len(results[i][0].boxes.conf)>1 and results[i][0].boxes.conf[1] > 0.5)):
    phase4_count = phase4_count + 1
    if phase4_count>=limit:
      flag4 = True
      phase4_idx = count

  elif flag4 == False and flag5 == False and 1.0 not in results[i][0].boxes.cls:
    phase4_count = 0

  if flag4 == True and flag5 == True and flag6 == False and 0.0 not in results[i][0].boxes.cls:
    phase6_count +=1
    if phase6_count>=limit:
      flag6 = True
      phase6_idx = count
  elif flag4 == True and flag5 == True and flag6 == False and 0.0 in results[i][0].boxes.cls:
    phase6_count = 0
  count+=1


print(phase4_idx, phase5_idx)



100%|██████████| 319/319 [00:00<00:00, 11912.67it/s]

95 189


In [32]:
print(phase1_idx, phase2_idx, phase3_idx, phase4_idx, phase5_idx, phase6_idx)

1 60 63 95 189 263


In [33]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/ETVProject/ETV_Project/detect/train/weights/best.pt' conf=0.5 source='/content/drive/MyDrive/ETVProject/ETV_Project/chiari_predictions3' save=True

2023-11-09 07:19:21.381775: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 07:19:21.381839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 07:19:21.381887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 07:19:23.190895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
image 1/319 /content/drive/MyDrive/ETVProject/ETV_Project/

In [34]:
imlist = natsorted(os.listdir("/content/runs/detect/predict/"))

In [35]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont


In [36]:
count = 0
for f in tqdm(imlist):
  if '.png' in f:
    filename = "/content/runs/detect/predict/"+f
    img = Image.open(filename)
    I1 = ImageDraw.Draw(img)

    if count >=phase6_idx:
      I1.text((10, 10), "5 Phases Complete", fill =(255, 0, 0))
    elif count >= phase5_idx:
      I1.text((10, 10), "Dilation", fill =(255, 0, 0))
    elif count >= phase4_idx:
      I1.text((10, 10), "Puncturing", fill =(255, 0, 0))
    elif count >= phase3_idx:
      I1.text((10, 10), "Third Ventricle", fill =(255, 0, 0))
    elif count >= phase2_idx:
      I1.text((10, 10), "Entering Third Ventricle", fill =(255, 0, 0))
    elif count >= phase1_idx:
      I1.text((10, 10), "Lateral Ventricle", fill =(255, 0, 0))
    #img.show()
    img=img.save(filename)
    count+=1

100%|██████████| 319/319 [00:04<00:00, 71.49it/s]


In [37]:
import cv2
import glob

In [38]:
frameSize = (224, 224)
out = cv2.VideoWriter('/content/drive/MyDrive/ETVProject/ETV_Project/chiari_opt.avi',cv2.VideoWriter_fourcc(*'DIVX'), 2, frameSize)

In [39]:
test_images = natsorted(glob.glob("/content/runs/detect/predict/*.png"))

In [40]:
for filename in tqdm(test_images):
    img = cv2.imread(filename)
    out.write(img)


100%|██████████| 319/319 [00:00<00:00, 585.35it/s]


In [41]:

out.release()